In [16]:
#CARGA DE LIBRERIAS
import pandas as pd
import pickle
import gzip
import os
import json
import numpy as np 
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score, confusion_matrix

In [2]:
#LECTURA DE DATOS
train_data_zip = '../files/input/train_data.csv.zip'
test_data_zip = '../files/input/test_data.csv.zip'

In [3]:
#DESCOMPRIMIR ARCHIVOS 
train_data = pd.read_csv(train_data_zip, index_col=False, compression='zip')
test_data = pd.read_csv(test_data_zip, index_col=False, compression='zip')

In [5]:
train_data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,10748,310000,1,3,1,32,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,12574,10000,2,3,1,49,-1,-1,-2,-1,...,1690,1138,930,0,0,2828,0,182,0,1
2,29677,50000,1,2,1,28,-1,-1,-1,0,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,8857,80000,2,3,1,52,2,2,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,21099,270000,1,1,2,34,1,2,0,0,...,22448,15490,17343,0,4000,2000,0,2000,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,27956,140000,2,2,1,27,2,-1,-1,-1,...,1580,804,728,752,800,1580,0,700,700,0
20996,27108,130000,1,2,2,41,0,0,0,0,...,123107,42897,39378,4442,5200,5012,2500,5000,2000,0
20997,26,50000,1,3,2,23,0,0,0,0,...,28967,29829,30046,1973,1426,1001,1432,1062,997,0
20998,14778,90000,2,3,2,25,0,0,0,0,...,5613,10113,10113,3000,3000,0,4500,0,3440,0


In [6]:
test_data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0
2,11,200000,2,3,2,34,0,0,2,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,15,250000,1,1,2,29,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,16,50000,2,3,3,23,1,2,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,29984,20000,1,2,1,44,-2,-2,-2,-2,...,2882,9235,1719,2890,2720,2890,9263,1824,1701,0
8996,29987,360000,1,1,2,35,-1,-1,-2,-2,...,0,0,0,0,0,0,0,0,0,0
8997,29990,150000,1,1,2,35,-1,-1,-1,-1,...,780,0,0,9054,0,783,0,0,0,0
8998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1


In [7]:
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          21000 non-null  int64
 1   LIMIT_BAL                   21000 non-null  int64
 2   SEX                         21000 non-null  int64
 3   EDUCATION                   21000 non-null  int64
 4   MARRIAGE                    21000 non-null  int64
 5   AGE                         21000 non-null  int64
 6   PAY_0                       21000 non-null  int64
 7   PAY_2                       21000 non-null  int64
 8   PAY_3                       21000 non-null  int64
 9   PAY_4                       21000 non-null  int64
 10  PAY_5                       21000 non-null  int64
 11  PAY_6                       21000 non-null  int64
 12  BILL_AMT1                   21000 non-null  int64
 13  BILL_AMT2                   21000 non-null  int64
 14  BILL_A

In [11]:
# Paso 1.
# Realice la limpieza de los datasets:
# - Renombre la columna "default payment next month" a "default".
# - Remueva la columna "ID".
# - Elimine los registros con informacion no disponible.
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".

def clean_data(data_df):
    
    df=data_df.copy()
    df=df.rename(columns={'default payment next month': 'default'})
    df=df.drop(columns='ID')
    df=df.dropna()
    df.loc[:, 'EDUCATION'] = df['EDUCATION'].apply(lambda x: x if x <= 4 else 4)
    return df

In [12]:
train_data = clean_data(train_data)
test_data = clean_data(test_data)

In [14]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
x_train = train_data.drop(columns='default', axis=1)
y_train = train_data['default']

x_test = test_data.drop(columns='default', axis=1)
y_test = test_data['default']

In [15]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las demas variables al intervalo [0, 1].
# - Selecciona las K mejores caracteristicas.
# - Ajusta un modelo de regresion logistica.

cat_features_train = x_train.select_dtypes(include=["object", "category"]).columns
cat_features_test = x_test.select_dtypes(include=["object", "category"]).columns
cat_features = list(set(cat_features_train) & set(cat_features_test))


def create_pipeline(df):
    cat_features = ['SEX', 'EDUCATION', 'MARRIAGE']
    num_features = [col for col in df.columns if col not in cat_features]

    # Definir los transformadores
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), num_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
        ]
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('select_k_best', SelectKBest(f_classif)),
        ('model', LogisticRegression())
    ])

    return pipeline

In [17]:
pipeline = create_pipeline(x_train)

In [18]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.

def optimize_hyperparameters(pipeline, x_train, y_train):
    param_grid = {
        'model__C': [0.7, 0.8, 0.9],
        'model__solver': ['liblinear','saga'],
        'model__max_iter': [1500],
        'select_k_best__k': [1, 2, 5, 10]
    }
    grid_search=GridSearchCV(pipeline, param_grid, cv=10, scoring='balanced_accuracy', verbose=1, n_jobs=-1)
    grid_search.fit(x_train, y_train)
    return grid_search

In [19]:
model = optimize_hyperparameters(pipeline, x_train, y_train)
print(model.best_params_)

Fitting 10 folds for each of 24 candidates, totalling 240 fits
{'model__C': 0.7, 'model__max_iter': 1500, 'model__solver': 'liblinear', 'select_k_best__k': 1}


In [20]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

def save_model(model):
    
    if not os.path.exists('../files/models'):
        os.makedirs('../files/models')
    
    with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
        pickle.dump(model, file)

In [21]:
save_model(model)

In [22]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'type': 'metrics', 'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

def calculate_metrics(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': float(round(precision_score(y_train, y_train_pred),4)),
        'balanced_accuracy': float(round(balanced_accuracy_score(y_train, y_train_pred),4)),
        'recall': float(round(recall_score(y_train, y_train_pred),4)),
        'f1_score': float(round(f1_score(y_train, y_train_pred),4))
    }

    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'precision': float(round(precision_score(y_test, y_test_pred),4)),
        'balanced_accuracy': float(round(balanced_accuracy_score(y_test, y_test_pred),4)),
        'recall': float(round(recall_score(y_test, y_test_pred),4)),
        'f1_score': float(round(f1_score(y_test, y_test_pred),4))
    }

    print(metrics_train)
    print(metrics_test)

    return metrics_train, metrics_test

In [23]:

metrics_train, metrics_test = calculate_metrics(model, x_train, y_train, x_test, y_test)

{'type': 'metrics', 'dataset': 'train', 'precision': 0.6933, 'balanced_accuracy': 0.6392, 'recall': 0.3194, 'f1_score': 0.4374}
{'type': 'metrics', 'dataset': 'test', 'precision': 0.7006, 'balanced_accuracy': 0.6546, 'recall': 0.3494, 'f1_score': 0.4663}


In [24]:
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#
def calculate_confusion_matrix(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    cm_train = confusion_matrix(y_train, y_train_pred)
    cm_test = confusion_matrix(y_test, y_test_pred)

    cm_matrix_train = {
        'type': 'cm_matrix',
        'dataset': 'train',
        'true_0': {"predicted_0": int(cm_train[0, 0]), "predicted_1": int(cm_train[0, 1])},
        'true_1': {"predicted_0": int(cm_train[1, 0]), "predicted_1": int(cm_train[1, 1])}
    }

    cm_matrix_test = {
        'type': 'cm_matrix',
        'dataset': 'test',
        'true_0': {"predicted_0": int(cm_test[0, 0]), "predicted_1": int(cm_test[0, 1])},
        'true_1': {"predicted_0": int(cm_test[1, 0]), "predicted_1": int(cm_test[1, 1])}
    }

    return cm_matrix_train, cm_matrix_test

In [26]:
cm_matrix_train, cm_matrix_test = calculate_confusion_matrix(model, x_train, y_train, x_test, y_test)

In [27]:
cm_matrix_train

{'type': 'cm_matrix',
 'dataset': 'train',
 'true_0': {'predicted_0': 15605, 'predicted_1': 668},
 'true_1': {'predicted_0': 3217, 'predicted_1': 1510}}

In [28]:
#guardar los parametros en carpeta output
def save_metrics(metrics_train, metrics_test, cm_matrix_train, cm_matrix_test):
    
    if not os.path.exists('../files/output'):
        os.makedirs('../files/output')
    
    metrics = [metrics_train, metrics_test, cm_matrix_train, cm_matrix_test]
    pd.DataFrame(metrics).to_json('../files/output/metrics.json', orient='records', lines=True)


In [29]:
save_metrics(metrics_train, metrics_test, cm_matrix_train, cm_matrix_test)